<a href="https://colab.research.google.com/github/msan222/Response_Bot/blob/main/dataSetsFeed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
import gdown
#!pip install google.colab

In [ ]:
import os
!git clone --quiet https://github.com/pistocop/messaging-chat-parser.git
!pip install -q -r messaging-chat-parser/requirements.txt
!git clone --quiet https://github.com/pistocop/pistoBot.git



In [ ]:
gdown.download('https://drive.google.com/uc?id=1f4vOzkZfcqzEAo-yujV0KNpGXUsCeQ_W','test.txt',quiet=False)

#https://drive.google.com/file/d/1f4vOzkZfcqzEAo-yujV0KNpGXUsCeQ_W/view?usp=sharing

In [4]:
!mv test.txt ./messaging-chat-parser/data/chat_raw/whatsapp

In [5]:
whatsapp_user_name = "Maddie_22" # <--- your name, extracted from Whatsapp data
whatsapp_datetime_format = "%m/%d/%y, %H:%M %p" # <-- American format used (MDY)

In [ ]:
# Whatsapp
!pwd
!echo $path
print("> [WHATSAPP] start parsing...")
assert whatsapp_user_name is not None, "[!] Whatsapp user name not setted"
!cd messaging-chat-parser/ && python ./src/whatsapp_parser.py --session_token "<|endoftext|>" --user_name $whatsapp_user_name --time_format "$whatsapp_datetime_format"
print("> [WHATSAPP] parsing completed!\n\n")
print("----------------------------------")

# Join Telegram and Whatsapp data
!cd messaging-chat-parser && python ./src/joiner.py
training_data_lines = sum(1 for line in open('./messaging-chat-parser/data/chat_parsed/all-messages.txt'))
print(f"> [PARSER] Training file lines: {training_data_lines}")
print("----------------------------------")

# Check data size
if training_data_lines < 1000:
    print(f"[WARNING] attention insufficient training data ({training_data_lines} < 100K), it is recommended to export more chats")

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("imdb")

In [8]:
import torch
torch.cuda.empty_cache()
!pwd
from torch.utils.data import Dataset
class CustomTextDataset(Dataset):
  def __init__(self, file):
    with open(file,'r') as infile:
      self.text = [line for line in infile]
        
  def __len__(self):
    return len(self.text)

  def __getitem__(self, idx):        
    return {'text':self.text[idx], 'label': 0}


r_j_dataset = CustomTextDataset('./messaging-chat-parser/data/chat_parsed/user-messages.txt')

/content


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")
datasets = load_dataset('./messaging-chat-parser/data/chat_parsed')

In [ ]:
#datasets = {'train': r_j_dataset, 'test':r_j_dataset}
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="longest",  truncation=True)

tokenized_datasets = datasets.map(tokenize_function, batched=True,remove_columns=["text"])
# block_size = tokenizer.model_max_length
block_size = 128

def group_texts(examples):
    #print(examples.keys())
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy() #check predictions 
    return result
    
tokenized_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)


In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100)) 
small_eval_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100)) 
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["train"]

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("gpt2-medium")

In [ ]:
from transformers import TrainingArguments
torch.cuda.empty_cache()
training_args = TrainingArguments("test_trainer")
#print(str(training_args))
training_args.fp16 = True

from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset
)

trainer.train()

In [ ]:
chat = []
start_temperature = 0.1
max_temperature = 3.0

for _ in range(5):
    new_line = "[others] " + input("✍") + '\n'
    chat.append(new_line)
    #ipdb.set_trace(context=6)
    me_token = False
    temperature = start_temperature
    input_network = ' '.join(chat)
    #ipdb.set_trace(context=6)
    while not me_token:
        text = ai.generate(prompt=input_network, 
                           return_as_list=True, 
                           temperature=temperature)
        text = text[0] # batch of 1
        #ipdb.set_trace(context=6)
        text = text.split()
        chat_pos = len(chat)
        #ipdb.set_trace(context=6)
        network_reply = text[chat_pos]
        #ipdb.set_trace(context=6)
        
        if network_reply.startswith('[me]'):
            me_token = True
            network_reply = text[chat_pos] + '\n'
            chat.append(network_reply)
        else:
            if temperature >= max_temperature:
                raise RuntimeError("Max temperature reached")
            temperature += 0.1
            print(f'temperature exit: {temperature}')
    print('Chat:')
    print(chat)
    print('---------------------')
    